Neste exemplo treinamos e avaliamos um modelo preditivo usando **T-fold-CrossValidation**, ou Validação Cruzada com T Pastas

- técnica de aprendizagem usada: Naive Bayes
- tarefa supervisionada: classificação de vinhos
- métricas de avaliação: taxa de acerto, precisão, revocação, f1 e matriz de confusão

Importando os recursos necessários:
- numpy: biblioteca numérica
- sklearn: biblioteca de machine learning, as métricas de avaliação e o model_selection que nos permite executar validação cruzada

In [7]:
# Importa bibliotecas necessárias 
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn import model_selection
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.datasets import load_wine

Carregando a base de dados do problema, representada aqui por X e y, onde:
- X: array contendo N instâncias com M atributos (atributos de entrada do problema)
- y: array contendo o rótulo (atributo alvo) de cada instância em X


In [8]:
# Neste exemplo a base de dados é composta por 178 instâncias 
# e cada instância é representada por um vetor de 13 atributos

X, y = load_wine(return_X_y=True)
print("Formato de X: ", X.shape)
print("Formato de y: ", y.shape)



Formato de X:  (178, 13)
Formato de y:  (178,)


In [9]:
# Usando o GridSearch para buscar os melhores parâmetros para nosso modelo
from sklearn.model_selection import train_test_split, GridSearchCV

# Separando da base original uma base de validação
X, X_val, y, y_val = train_test_split(X,y, test_size=0.2, random_state=42, stratify=y)

# Definindo a técnica a ser utilizada
clf = GaussianNB()

# parâmetros que o Gridsearch utilizará em sua busca
parameters = [{'var_smoothing':[1e-09, 1e-03, 1e-06]}]

# Execução do Gridsearch 
gs=GridSearchCV(clf, parameters, scoring='accuracy', cv=5, n_jobs=-1)
gs.fit(X_val, y_val)

# Mostrando a tabela de resultados do GridSearch (opcional)
from tabulate import tabulate
import pandas as pd
df=gs.cv_results_
print(tabulate(df, headers='keys', tablefmt='psql'))
print("Melhores parâmetros encontrados: ", gs.best_params_)

# Recuperando os melhores resultados
clf=gs.best_estimator_


+-----------------+----------------+-------------------+------------------+-----------------------+--------------------------+---------------------+---------------------+---------------------+---------------------+---------------------+-------------------+------------------+-------------------+
|   mean_fit_time |   std_fit_time |   mean_score_time |   std_score_time |   param_var_smoothing | params                   |   split0_test_score |   split1_test_score |   split2_test_score |   split3_test_score |   split4_test_score |   mean_test_score |   std_test_score |   rank_test_score |
|-----------------+----------------+-------------------+------------------+-----------------------+--------------------------+---------------------+---------------------+---------------------+---------------------+---------------------+-------------------+------------------+-------------------|
|      0.0019207  |    0.000773289 |        0.00194259 |      0.00158506  |                 1e-09 | {'var_smooth

Neste ponto definimos a técnica de Machine Learning a ser utilizada e treinamos o modelo usando Validação Cruzada. No exemplo, um classificador do tipo Naive Bayes.  

Observer que, diferente do HOLDOUT, não usamos a função *fit()* e sim *model_selection.cross_val_score()*. Esta função irá treinar T modelos. A cada iteração treina usando T-1 pastas e testa com uma delas. Troca a pasta de teste a cada iteração. Ao final fornece como resultado acurácia média + desvio padrão.

OBS: protocolo muito mais robusto que o HOLDOUT, pois cada instância do problema será teste em alguma iteração.

In [10]:

# Usando a validação cruzada com 5 folds neste exemplo.
T=5 # número de pastas ou folds
result = model_selection.cross_val_score(clf, X, y, cv=T)


Avaliação do modelo abaixo:

- A variável *result* que criamos anteriormente já tem a taxa de acerto média e desvio padrão calculadas pela função *model_selection.cross_val_score()*.

- A função *model_selection.cross_val_predict()* retorna a classe para cada exemplo de teste.

- A função *model_selection.cross_val_predict()* com parâmetro '*method=predict_proba*' retorna a probabilidade de cada classe para cada exemplo de teste.

Considerando tp=true positivive, tn=true negative, fp=false positive, fn=false negative.

- A função *precision_score()*: calcula tp / (tp + fp)

- A função *recall_score()* calcula: tp / (tp + fn)

- A função *f1_score()* calcula a média harmônica entre *precision* e *recall*.

- A função *confusion_matrix()* recebe como entrada os rótulos do teste (y_test) e a predição do modelo (y_pred). Ela retorna uma matriz CxC onde C é a quandidade de classes. No exemplo C=3, logo uma matriz 3x3 onde na diagonal temos os acertos e nas demais posições as confusões entre as classes do problema. A matriz de confusão é usada para avaliar classificador apenas e muito importante para analisarmos os erros do nosso modelo (ou hipótese de solução para o problema).  


In [12]:

# Mostrando a acurácia média e desvio padrão.
print("\nCross Validation Results %d folds:" % T)
print("Mean Accuracy: %.5f" % result.mean())
print("Mean Std: %.5f" % result.std())

# Calculando a predição para cada exemplo de teste
y_pred = model_selection.cross_val_predict(clf, X, y, cv=T)

# Calculando para cada instância de teste a probabilidade de cada classe
predicted_proba=model_selection.cross_val_predict(clf, X, y, cv=T, method='predict_proba')

# Calculando a precisão na base de teste
precision=precision_score(y, y_pred, average='weighted')
print("Precision = %.3f " % precision)

# Calculando a revocação na base de teste
recall=recall_score(y, y_pred, average='weighted')
print("Recall = %.3f " % recall)

# Calculando f1 na base de teste
f1=f1_score(y, y_pred, average='weighted')
print("F1 = %.3f " % f1)

# Exemplo mostrando o resultado previsto para a primeira instância de teste
print("Primeira instância na base de teste foi considerada como da classe: %d" % y_pred[0])

# Exemplo abaixo mostrando para a primeira instância de teste a probabilidade de cada classe
print("Probabilidade de cada classe para a primeira instância: ", predicted_proba[0])

# Calculando a matriz de confusão
print("Matriz de Confusão:")
matrix = confusion_matrix(y, y_pred)
print(matrix)

# salvando o modelo 
from joblib import dump, load
with open("NB.mod", 'wb') as fo:  
    dump(clf, fo)


Cross Validation Results 5 folds:
Mean Accuracy: 0.97192
Mean Std: 0.03440
Precision = 0.973 
Recall = 0.972 
F1 = 0.972 
Primeira instância na base de teste foi considerada como da classe: 0
Probabilidade de cada classe para a primeira instância:  [9.99376150e-01 6.23849708e-04 4.07558723e-11]
Matriz de Confusão:
[[46  1  0]
 [ 0 54  3]
 [ 0  0 38]]
